In [3]:
import os

In [4]:
pwd

'c:\\Users\\JOY\\Desktop\\DL_Project\\Renel_Disease_Classification_Deep_Learning\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\JOY\\Desktop\\DL_Project\\Renel_Disease_Classification_Deep_Learning'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [8]:
from Renel_Disease_Classifier.constants import *
from Renel_Disease_Classifier.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifact_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        




In [18]:
import os 
import urllib.request as request
import zipfile
import gdown
from Renel_Disease_Classifier.logger import logging
from Renel_Disease_Classifier.utils.common import get_size
import sys

In [19]:
from Renel_Disease_Classifier.exception import RenelException

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)->str:
        logging.info("Entered the download_file method of Data_Ingestion class") 

        try:

            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            pref = "https://drive.google.com/uc?/export=download&id="
            gdown.download(pref+file_id,zip_download_dir,quiet=True)

            logging.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")


        except Exception as e:
            raise RenelException(e,sys)
        

    def extract_zip_file(self):

        logging.info("Entered the extract_zip_file method of Data_Ingestion class")
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        try:

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logging.info(f"Extracted zip file into {unzip_path}")
        except Exception as e:
            raise RenelException(e,sys)

In [15]:
from Renel_Disease_Classifier.exception import RenelException

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise RenelException(e,sys)